#### **Attention is All You Need**

**본 코드는 [나동빈님의 코드 실습](https://github.com/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/Attention_is_All_You_Need_Tutorial_(German_English).ipynb)을 참조한 코드 리뷰입니다.**

**데이터셋을 불러오는 코드에 오류가 있어 새로운 데이터셋을 사용했습니다. 새로운 데이터셋을 사용했기 때문에 max_length 미설정 시 index 오류가 납니다.   모델 구조는 나동빈님의 코드를 그대로 사용하였고, 클래스 정의부 안에 세부 설명을 추가하였습니다.**

In [ ]:
!pip install torchtext==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)


In [ ]:
%%capture
!pip install -U pip setuptools wheel

!pip install -U spacy

In [ ]:
!python -m spacy download ko_core_news_sm # 한국어 모듈 다운
!python -m spacy download en_core_web_sm # 영어 모듈 다운

In [ ]:
import spacy
import os

spacy_ko = spacy.load('ko_core_news_sm') # 한국어 토큰화(tokenization)
spacy_en = spacy.load('en_core_web_sm') # 영어 토큰화(tokenization)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 간단히 토큰화(tokenization) 기능 써보기
tokenized = spacy_en.tokenizer("Hello my name is Paul")
tokenized_ko = spacy_ko.tokenizer("안녕 나는 폴이야")
for i, token in enumerate(tokenized):
    print(f"인덱스 {i}: {token.text}")

for i, token in enumerate(tokenized_ko):
    print(f"인덱스 {i}: {token.text}")


*나동빈님의 코드를 그대로 사용한 부분입니다.*
* 영어(English) 및 한국어(Korean) **토큰화 함수** 정의

In [ ]:
# 한국어 문장을 토큰화 하는 함수 (순서를 뒤집지 않음)
def tokenize_ko(text):
    return [token.text for token in spacy_ko.tokenizer(text)]

# 영어 문장을 토큰화 하는 함수
def tokenize_en(text):
    return [token.text for token in spacy_en.tokenizer(text)]

*나동빈님의 코드를 그대로 사용한 부분입니다.*

In [ ]:
from torchtext.data import Field, BucketIterator

# Field 라이브러리 : 텍스트 전처리, 토큰화 등에 사용되는 기능을 가진 도구.
# 번역의 입력값인 source(SRC)는 한국어, 번역 결과인 target(TRG)는 영어 설정.
SRC = Field(tokenize=tokenize_ko, init_token="<sos>", eos_token="<eos>", lower=True, batch_first=True)
TRG = Field(tokenize=tokenize_en, init_token="<sos>", eos_token="<eos>", lower=True, batch_first=True)

In [ ]:
import pandas as pd
# load dataset
file_path = '/content/drive/MyDrive/Colab Notebooks/data/kr-en/'
# file_path = '../../data/kr-en/'

for idx,file in enumerate(os.listdir(file_path)):
    if file.endswith('.xlsx'):
        globals()[f"df{idx}"] = pd.read_excel(os.path.join(file_path, file))

df0.head(), df1.head()

In [ ]:
idx

In [ ]:
# 데이터프레임의 컬럼 확인 -> 한국어와 영어 문장 컬럼만 남길 것.
for i in range(idx):
    print(globals()[f"df{i}"].columns)
    print(globals()[f"df{i}"]['원문'][0])

In [ ]:
# 원문(한국어)은 src, 번역문(영어)은 trg으로 컬럼 이름 변경 후 하나의 데이터프레임으로 합치기
kr_en = pd.DataFrame({'src':[],'trg':[]})
for i in range(idx):
    globals()[f"df{i}"].rename(columns={'원문':'src', '번역문':'trg'}, inplace=True)
    kr_en = pd.concat([kr_en, globals()[f"df{i}"][['src','trg']]],axis=0)
print(kr_en.shape)
kr_en.head()

In [ ]:
# train, validation, test 데이터셋 분할
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(kr_en, test_size=0.15, random_state=42)
train_data, valid_data = train_test_split(train_data, test_size=0.15, random_state=42)

train_data.reset_index(drop=True,inplace=True)
valid_data.reset_index(drop=True,inplace=True)
test_data.reset_index(drop=True,inplace=True)

# 데이터가 너무 많으니 조금 줄인다.
train_data = train_data.iloc[:500000]
valid_data = valid_data.iloc[:70000]
test_data = test_data.iloc[:70000]

print(f"학습 데이터셋(training dataset) 크기: {len(train_data)}개")
print(f"평가 데이터셋(validation dataset) 크기: {len(valid_data)}개")
print(f"테스트 데이터셋(testing dataset) 크기: {len(test_data)}개")

In [ ]:
### 데이터셋의 Max_Length 구하기
# 모든 셀의 문장 길이 계산
max_length = 0
for col in ['src', 'trg']:
    for sentence in train_data[col]:
        words = sentence.split()
        length = len(words)
        if length > max_length:
            max_length = length
for col in ['src', 'trg']:
    for sentence in valid_data[col]:
        words = sentence.split()
        length = len(words)
        if length > max_length:
            max_length = length
for col in ['src', 'trg']:
    for sentence in test_data[col]:
        words = sentence.split()
        length = len(words)
        if length > max_length:
            max_length = length
print(max_length)

In [ ]:
# 데이터셋 분할을 위해 TabularDataset 사용하기 위해 kr_en 데이터셋을 다시 파일로 저장한 후 불러온다.
train_data.to_csv(os.path.join(file_path,'train.csv'),index=False)
valid_data.to_csv(os.path.join(file_path,'valid.csv'),index=False)
test_data.to_csv(os.path.join(file_path,'test.csv'),index=False)

### train, valid, test.csv를 kr-en/ 디렉토리에 저장하기 때문에 만약 코드를 처음부터 다시 시작한다면
# for idx,file in enumerate(os.listdir(file_path)): > 여기 데이터셋 로드하는 코드에서
# idx가 저 csv파일까지도 포함해서 에러 날 수 있으니 train,valid,test.csv를 삭제하고 다시 시작해주세요.

from torchtext.data import Field, TabularDataset, BucketIterator
train_dataset, valid_dataset, test_dataset = TabularDataset.splits(
    path=file_path,
    train='train.csv',
    validation='valid.csv',
    test='test.csv',
    format='csv',
    fields=[('src',SRC), ('trg',TRG)]
)
# 데이터셋 확인
print(f'Number of training examples: {len(train_dataset.examples)}')
print(f'Number of validation examples: {len(valid_dataset.examples)}')
print(f'Number of testing examples: {len(test_dataset.examples)}')
print(vars(train_dataset.examples[1])['src'])
print(vars(train_dataset.examples[1])['trg'])

In [ ]:
vars(train_dataset.examples[0])

*나동빈님의 코드를 그대로 사용한 부분입니다.*

* **필드(field)** 객체의 **build_vocab** 메서드를 이용해 영어와 한국어의 단어 사전을 생성합니다.
  * **최소 2번 이상** 등장한 단어만을 선택합니다.

In [ ]:
SRC.build_vocab(train_dataset, min_freq=2)
TRG.build_vocab(train_dataset, min_freq=2)

print(f"len(SRC): {len(SRC.vocab)}")
print(f"len(TRG): {len(TRG.vocab)}")

*나동빈님의 코드를 그대로 사용한 부분입니다.*

In [ ]:
print(TRG.vocab.stoi["abcabc"]) # 없는 단어: 0
print(TRG.vocab.stoi[TRG.pad_token]) # 패딩(padding): 1
print(TRG.vocab.stoi["<sos>"]) # <sos>: 2
print(TRG.vocab.stoi["<eos>"]) # <eos>: 3
print(TRG.vocab.stoi["hello"])
print(TRG.vocab.stoi["need"])

*나동빈님의 코드를 그대로 사용한 부분입니다.*
* 한 문장에 포함된 단어가 순서대로 나열된 상태로 네트워크에 입력되어야 합니다.
    * 따라서 하나의 배치에 포함된 문장들이 가지는 단어의 개수가 유사하도록 만들면 좋습니다.
    * 이를 위해 BucketIterator를 사용합니다.
    * **배치 크기(batch size)**: 128

[comment] <br>
iterator들에는 문장들이 다음과 같이 저장된다. <br>
각 batch에는 128개의 문장들이 들어있다. <br>
각 batch에 들어있는 128개의 문장들은 batch.src.shape[1]의 값인 src_len 길이의 토큰으로 구성된 문장들이다.<br>
즉, batch.src.shape[1]값이 36이면 36개의 토큰으로 구성된 서로 다른 문장들이 하나의 batch에 128개 있다는 것이다. <br>
그리고 각 batch마다 src_len 값은 다를 수 있다.<br>
하나의 batch 안에 당연히 src_len길이에 해당하는 문장들만 있는 것은 아니고, src_len보다 짧은 문장들도 있다. 이런 문장들은 남은 길이의 토큰을 \<pad>토큰으로 채우게 되고 \<pad> 토큰의 인덱스값은 1이다.

In [ ]:
import torch
# CUDA_LAUNCH_BLOCKING=1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

BATCH_SIZE = 128

# 일반적인 데이터 로더(data loader)의 iterator와 유사하게 사용 가능
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_dataset, valid_dataset, test_dataset),
    batch_size=BATCH_SIZE,
    sort=False,
    device=device)

for i, batch in enumerate(train_iterator):
    src = batch.src
    trg = batch.trg
    print(src[0].shape)
    print(f"첫 번째 배치 크기: {src.shape}")

    for b in range(src.shape[0]):
        print('batch :',b)
        if b == 2:
            break
        for i in range(src.shape[1]):
            print(f"인덱스 {i}: {src[b][i].item()}", end=" ") # 여기에서는 [Seq_num, Seq_len]

    break
print()

      
*🫡아래부터는 모두 나동빈님의 코드를 그대로 사용한 부분입니다. 클래스 정의부에 구체적인 설명을 추가했습니다.*
#### **Multi Head Attention 아키텍처**

* 어텐션(attention)은 <b>세 가지 요소</b>를 입력으로 받습니다.
    * <b>쿼리(queries)</b>
    * <b>키(keys)</b>
    * <b>값(values)</b>
    * 현재 구현에서는 Query, Key, Value의 차원이 모두 같습니다.
* 하이퍼 파라미터(hyperparameter)
    * **hidden_dim**: 하나의 단어에 대한 임베딩 차원
    * **n_heads**: 헤드(head)의 개수 = scaled dot-product attention의 개수
    * **dropout_ratio**: 드롭아웃(dropout) 비율

In [ ]:
import torch.nn as nn

class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, dropout_ratio, device='cpu'):
        super().__init__()

        assert hidden_dim % n_heads == 0

        self.hidden_dim = hidden_dim # 임베딩 차원
        self.n_heads = n_heads # 헤드(head)의 개수: 서로 다른 어텐션(attention) 컨셉의 수
        self.head_dim = hidden_dim // n_heads # 각 헤드(head)에서의 임베딩 차원

        self.fc_q = nn.Linear(hidden_dim, hidden_dim) # Query 값에 적용될 FC 레이어
        self.fc_k = nn.Linear(hidden_dim, hidden_dim) # Key 값에 적용될 FC 레이어
        self.fc_v = nn.Linear(hidden_dim, hidden_dim) # Value 값에 적용될 FC 레이어

        self.fc_o = nn.Linear(hidden_dim, hidden_dim)

        self.dropout = nn.Dropout(dropout_ratio)

        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)

    def forward(self, query, key, value, mask = None):

        batch_size = query.shape[0]

        # query_len, key_len, value_len 모두 src_len과 동일.
        # query: [batch_size, query_len, hidden_dim]
        # key: [batch_size, key_len, hidden_dim]
        # value: [batch_size, value_len, hidden_dim]

        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)

        # Q: [batch_size, query_len, hidden_dim]
        # K: [batch_size, key_len, hidden_dim]
        # V: [batch_size, value_len, hidden_dim]

        # hidden_dim → n_heads X head_dim 형태로 변형
        # n_heads(h)개의 서로 다른 어텐션(attention) 컨셉을 학습하도록 유도
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3) # -1 = query_len
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        # tensor_obj.permute(0,2,1,3) >> 기존에 [0,1,2,3] 순서였던 데이터를 [0,2,1,3]으로 차원을 변형한 것.

        # Q: [batch_size, n_heads, query_len, head_dim]
        # K: [batch_size, n_heads, key_len, head_dim]
        # V: [batch_size, n_heads, value_len, head_dim]

        # Attention Energy 계산
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        # 3차원 이상의 행렬을 행렬곱할 때도 2차원과 마찬가지로 마지막 두 차원의 행렬곱만 맞춰주면 된다.
        '''
        shape이 (3,4,5)인 행렬 A와 (3,5,4)인 행렬 B는 뒤의 2개 차원이 (4,5) x (5,4)로 행렬곱이 가능하므로 AxB가 가능하다. >> 결과는 (3,4,4)가 된다.
        Q: [batch_size, n_heads, query_len, head_dim]
        K: [batch_size, n_heads, key_len, head_dim]
        두 행렬을 곱하려면 두 행렬의 뒤의 두 차원을 맞춰줘야 한다.
        K를 [batch_size, n_heads, head_dim, key_len] 이렇게 바꿔주면 되므로 permute(0,1,3,2)로 뒤의 두 차원을 변경해준다.
        '''
        # Attention을 계산할 때, 논문에서는 sqrt(K_dim)으로 나누어 attention energy 값을 정규화하는 역할을 했다.
        # MultiHead이므로 각 Head에서의 K_dim 은 hidden_dim // n_heads가 되기 때문에 self.scale은 sqrt(hidden_dim//n_heads)인 것이다.

        # energy: [batch_size, n_heads, query_len, key_len]

        # 마스크(mask)를 사용하는 경우
        if mask is not None:
            # 마스크(mask) 값이 0인 부분을 -1e10으로 채우기
            energy = energy.masked_fill(mask==0, -1e10)
            '''
            src_mask의 shape이 (batch_size, 1, 1, src_len)이었다.
            energy.shape = [batch_size, n_heads, query_len, key_len]이므로
            이는 src_mask에서 0인 토큰의 key_value를 '의미없음'을 나타내는 매우 작은 값으로 변경하는 코드이다.
            '''

        # 어텐션(attention) 스코어 계산: 각 단어에 대한 확률 값
        attention = torch.softmax(energy, dim=-1) # softmax 활성화 함수를 통해 각 energy 값을 확률 형태로 바꿔준다.

        # attention: [batch_size, n_heads, query_len, key_len]

        # 여기에서 Scaled Dot-Product Attention을 계산
        x = torch.matmul(self.dropout(attention), V)

        # x: [batch_size, n_heads, query_len, head_dim]

        x = x.permute(0, 2, 1, 3).contiguous()
        # permute()하는 이유 : 다시 [batch_size, src_len, hidden_dim]으로 output을 출력하기 위함.
        # contiguous() : torch 연산으로 인해 같은 배열의 원소들이 비연속적인 메모리에 저장될 수 있으므로 이를 해결하기 위해 연속적인 메모리 레이아웃에 텐서를 다시 할당하는 것.

        # x: [batch_size, query_len, n_heads, head_dim]

        x = x.view(batch_size, -1, self.hidden_dim)

        # x: [batch_size, query_len, hidden_dim]

        x = self.fc_o(x)

        # x: [batch_size, query_len, hidden_dim]

        return x, attention

#### **Position-wise Feedforward 아키텍처**

* 입력과 출력의 차원이 동일합니다.
* 하이퍼 파라미터(hyperparameter)
    * **hidden_dim**: 하나의 단어에 대한 임베딩 차원
    * **pf_dim**: Feedforward 레이어에서의 내부 임베딩 차원
    * **dropout_ratio**: 드롭아웃(dropout) 비율

In [ ]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hidden_dim, pf_dim, dropout_ratio):
        super().__init__()

        self.fc_1 = nn.Linear(hidden_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hidden_dim)

        self.dropout = nn.Dropout(dropout_ratio)

    def forward(self, x):

        # x: [batch_size, seq_len, hidden_dim]

        x = self.dropout(torch.relu(self.fc_1(x)))

        # x: [batch_size, seq_len, pf_dim]

        x = self.fc_2(x)

        # x: [batch_size, seq_len, hidden_dim]

        return x

#### **인코더(Encoder) 레이어 아키텍처**

* 하나의 인코더 레이어에 대해 정의합니다.
    * 입력과 출력의 차원이 같습니다.
    * 이러한 특징을 이용해 트랜스포머의 인코더는 인코더 레이어를 여러 번 중첩해 사용합니다.
* 하이퍼 파라미터(hyperparameter)
    * **hidden_dim**: 하나의 단어에 대한 임베딩 차원
    * **n_heads**: 헤드(head)의 개수 = scaled dot-product attention의 개수
    * **pf_dim**: Feedforward 레이어에서의 내부 임베딩 차원
    * **dropout_ratio**: 드롭아웃(dropout) 비율
* &lt;pad&gt; 토큰에 대하여 마스크(mask) 값을 0으로 설정합니다.

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, pf_dim, dropout_ratio, device='cpu'):
        super().__init__()

        self.self_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.ff_layer_norm = nn.LayerNorm(hidden_dim)
        self.self_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hidden_dim, pf_dim, dropout_ratio)
        self.dropout = nn.Dropout(dropout_ratio)

    # 하나의 임베딩이 복제되어 Query, Key, Value로 입력되는 방식
    def forward(self, src, src_mask):

        # src: [batch_size, src_len, hidden_dim]
        # ⚠️src_mask: [batch_size, src_len] ?? NO >> src_mask: [batch_size, 1, 1, src_len]

        # self attention
        # 필요한 경우 마스크(mask) 행렬을 이용하여 어텐션(attention)할 단어를 조절 가능
        _src, _ = self.self_attention(src, src, src, src_mask) # Query, Key, Value, Mask

        # dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        '''
        nn.LayerNorm() : 레이어 정규화(Layer Normalization) 클래스, 레이어 정규화는 신경망의 각 레이어의 출력을 정규화하여 훈련을 안정화시키고,
        학습 속도를 향상시키는 데 사용된다. 이는 배치 정규화(Batch Normalization)와 비슷한 목적을 가지고 있지만,
        배치 정규화는 배치 차원에 대해 정규화하는 반면, 레이어 정규화는 입력 텐서의 각 hidden 차원에 대해 정규화한다.
        '''
        # src + self.dropout(_src) : Residual connection으로 기울기 소실(vanishing gradient) 문제를 완화하고, 학습을 더 효과적으로 수행할 수 있도록 돕는 기술

        # src: [batch_size, src_len, hidden_dim]

        # position-wise feedforward
        _src = self.positionwise_feedforward(src)
        # 논문에 소개됐듯이 비선형성을 추가하고 더 구체적인 특징을 추출하고자 fc_layer를 2번 거치는 레이어가 추가됨.

        # dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))
        # 다시 한 번 residual connection과 레이어 정규화 진행

        # src: [batch_size, src_len, hidden_dim]

        return src

*nn.Embedding()클래스에 대한 설명은 [블로그](https://song9ski-program.tistory.com/entry/Pytorch-nnEmbedding%EA%B3%BC-BertTokenizer%EB%A5%BC-%ED%99%9C%EC%9A%A9%ED%95%9C-%EC%9E%84%EB%B2%A0%EB%94%A9#%F0%9F%94%B8nn.Embedding%EC%9D%98%20%EC%A3%BC%EC%9A%94%20%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0-1)를 참고하세요.*

#### **인코더(Encoder) 아키텍처**

* 전체 인코더 아키텍처를 정의합니다.
* 하이퍼 파라미터(hyperparameter)
    * **input_dim**: 하나의 단어에 대한 원 핫 인코딩 차원
    * **hidden_dim**: 하나의 단어에 대한 임베딩 차원
    * **n_layers**: 내부적으로 사용할 인코더 레이어의 개수
    * **n_heads**: 헤드(head)의 개수 = scaled dot-product attention의 개수
    * **pf_dim**: Feedforward 레이어에서의 내부 임베딩 차원
    * **dropout_ratio**: 드롭아웃(dropout) 비율
    * **max_length**: 문장 내 최대 단어 개수
* 원본 논문과는 다르게 <b>위치 임베딩(positional embedding)을 학습</b>하는 형태로 구현합니다.
    * BERT와 같은 모던 트랜스포머 아키텍처에서 사용되는 방식입니다.
* &lt;pad&gt; 토큰에 대하여 마스크(mask) 값을 0으로 설정합니다.

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio, device='cpu', max_length=100):
        super().__init__()

        self.device = device

        self.tok_embedding = nn.Embedding(input_dim, hidden_dim)
        self.pos_embedding = nn.Embedding(max_length, hidden_dim)

        self.layers = nn.ModuleList([EncoderLayer(hidden_dim, n_heads, pf_dim, dropout_ratio, device) for _ in range(n_layers)])

        self.dropout = nn.Dropout(dropout_ratio)

        self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)

    def forward(self, src, src_mask):

        # src: [batch_size, src_len]
        # ⚠️src_mask: [batch_size, src_len] ?? NO >> src_mask: [batch_size, 1, 1, src_len]

        batch_size = src.shape[0]
        src_len = src.shape[1]

        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        # torch.arange(start,end) : start부터 end-1까지의 정수값을 생성 np.arange()와 동일.
        # unsqueeze(0) : torch.arange(0,src_len)으로 [0,...,src_len-1]처럼 (src_len,)shape의 1차원 배열을 (1,src_len)으로 2차원으로 늘림.
        # tensor_obj.repeat(x,y) : tensor_obj를 행으로 x만큼, 열로 y만큼 복제한다. >> repeat(batch_size,1)은 torch_obj를 행으로 torch.concat(,dim=0)을 batch_size만큼 한 것. >> (1,src_len)이 (128,src_len)으로 변함.

        # pos: [batch_size, src_len]
        '''
        pos = [
            [0,1,2,3,...,src_len-1까지],
            ... batch_size행만큼 반복
            [0,1,2,3,...,src_len-1까지]
        ]
        '''

        # 소스 문장의 임베딩과 위치 임베딩을 더한 것을 사용
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        # self.tok_embedding(src)의 output shape은 (batch_size, src_len, hidden_dim) 이다.
        '''
        nn.Embedding에서는 len(SRC.vocab) * hidden_dim 사이즈의 lookup table을 내부적으로 학습하게 되고,
        tok_embedding(src) > src의 토큰들이 들어가게 되면 lookup table에서 각 토큰의 임베팅 벡터가 출력되어
        output shape이 (batch_size, src_len, hidden_dim)이다.
        '''
        # self.scale을 곱하는 것 : self.scale의 값은 hidden_dim의 제곱근이다. > tok_embedding결과값들에 self.scale을 곱함으로써 임베딩 값의 분포를 조절하고 학습을 안정화시키는 효과를 얻을 수 있다고 한다.
        # self.pos_embedding(pos)의 output shape도 (batch_size, src_len, hidden_dim) 이다.
        '''
        나동빈님은 positional encoding 대신 문장에서 토큰의 위치도 학습시켜 인코딩하는 방식을 선택하셨다.
        이에 따라 pos_embedding에서도 max_length * hidden_dim 사이즈의 lookup table이 내부적으로 학습되고,
        각 토큰들은 자신의 위치에 맞는 임베딩 벡터와 대응되어 출력된다.
        따라서 output shape이 (batch_size, src_len, hidden_dim)이 된다.
        '''
        # self.dropout() : 입력 텐서의 일부 요소를 무작위로 0으로 만든다.

        # src: [batch_size, src_len, hidden_dim]

        # 모든 인코더 레이어를 차례대로 거치면서 순전파(forward) 수행
        for layer in self.layers:
            src = layer(src, src_mask)

        # src: [batch_size, src_len, hidden_dim]

        return src # 마지막 레이어의 출력을 반환

#### **디코더(Decoder) 레이어 아키텍처**

* 하나의 디코더 레이어에 대해 정의합니다.
    * 입력과 출력의 차원이 같습니다.
    * 이러한 특징을 이용해 트랜스포머의 디코더는 디코더 레이어를 여러 번 중첩해 사용합니다.
    * 디코더 레이어에서는 두 개의 Multi-Head Attention 레이어가 사용됩니다.
* 하이퍼 파라미터(hyperparameter)
    * **hidden_dim**: 하나의 단어에 대한 임베딩 차원
    * **n_heads**: 헤드(head)의 개수 = scaled dot-product attention의 개수
    * **pf_dim**: Feedforward 레이어에서의 내부 임베딩 차원
    * **dropout_ratio**: 드롭아웃(dropout) 비율
* 소스 문장의 &lt;pad&gt; 토큰에 대하여 마스크(mask) 값을 0으로 설정합니다.
* 타겟 문장에서 각 단어는 다음 단어가 무엇인지 알 수 없도록(이전 단어만 보도록) 만들기 위해 마스크를 사용합니다.

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, pf_dim, dropout_ratio, device='cpu'):
        super().__init__()

        self.self_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.ff_layer_norm = nn.LayerNorm(hidden_dim)
        self.self_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
        self.encoder_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hidden_dim, pf_dim, dropout_ratio)
        self.dropout = nn.Dropout(dropout_ratio)

    # 인코더의 출력 값(enc_src)을 어텐션(attention)하는 구조
    def forward(self, trg, enc_src, trg_mask, src_mask):

        # trg: [batch_size, trg_len, hidden_dim]
        # enc_src: [batch_size, src_len, hidden_dim]
        # trg_mask: [batch_size, 1, trg_len, trg_len]
        # src_mask: [batch_size, 1, 1, src_len]

        # self attention
        # 자기 자신에 대하여 어텐션(attention)
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)
        # EncoderLayer에서의 self_attention() 과정과 동일하다.

        # dropout, residual connection and layer norm
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
        # 마찬가지로 EncoderLayer에서의 residual connection과 layer normalization과 동일하다.

        # trg: [batch_size, trg_len, hidden_dim]

        # encoder attention : encoder에서 구한 key, value와 decoder의 query에 대해 attention mechanism을 적용한다.
        # 디코더의 쿼리(Query)를 이용해 인코더를 어텐션(attention)
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)
        # key, value 파라미터 자리에 encoder에서 구한 key, value 값(=enc_src)이 전달된다.

        # dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))
        # 마찬가지로 EncoderLayer에서의 residual connection과 layer normalization과 동일하다.

        # trg: [batch_size, trg_len, hidden_dim]

        # positionwise feedforward
        _trg = self.positionwise_feedforward(trg)
        # 마찬가지로 EncoderLayer에서의 positionwise_feedforward와 동일하다.

        # dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))
        # 마찬가지로 EncoderLayer에서의 ff_layer_norm과 동일하다.

        # trg: [batch_size, trg_len, hidden_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

        return trg, attention

#### **디코더(Decoder) 아키텍처**

* 전체 디코더 아키텍처를 정의합니다.
* 하이퍼 파라미터(hyperparameter)
    * **output_dim**: 하나의 단어에 대한 원 핫 인코딩 차원
    * **hidden_dim**: 하나의 단어에 대한 임베딩 차원
    * **n_layers**: 내부적으로 사용할 인코더 레이어의 개수
    * **n_heads**: 헤드(head)의 개수 = scaled dot-product attention의 개수
    * **pf_dim**: Feedforward 레이어에서의 내부 임베딩 차원
    * **dropout_ratio**: 드롭아웃(dropout) 비율
    * **max_length**: 문장 내 최대 단어 개수
* 원본 논문과는 다르게 <b>위치 임베딩(positional embedding)을 학습</b>하는 형태로 구현합니다.
    * BERT와 같은 모던 트랜스포머 아키텍처에서 사용되는 방식입니다.
* Seq2Seq과는 마찬가지로 실제로 추론(inference) 시기에서는 디코더를 반복적으로 넣을 필요가 있습니다.
    * 학습(training) 시기에서는 한 번에 출력 문장을 구해 학습할 수 있습니다.
* 소스 문장의 &lt;pad&gt; 토큰에 대하여 마스크(mask) 값을 0으로 설정합니다.
* 타겟 문장에서 각 단어는 다음 단어가 무엇인지 알 수 없도록(이전 단어만 보도록) 만들기 위해 마스크를 사용합니다.

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio, device='cpu', max_length=100):
        super().__init__()

        self.device = device

        self.tok_embedding = nn.Embedding(output_dim, hidden_dim)
        self.pos_embedding = nn.Embedding(max_length, hidden_dim)

        self.layers = nn.ModuleList([DecoderLayer(hidden_dim, n_heads, pf_dim, dropout_ratio, device) for _ in range(n_layers)])

        self.fc_out = nn.Linear(hidden_dim, output_dim)

        self.dropout = nn.Dropout(dropout_ratio)

        self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)

    def forward(self, trg, enc_src, trg_mask, src_mask):

        # trg: [batch_size, trg_len]
        # enc_src: [batch_size, src_len, hidden_dim]
        # trg_mask : [batch_size, 1, trg_len, trg_len]
        # src_mask: [batch_size, 1, 1, src_len]

        batch_size = trg.shape[0]
        trg_len = trg.shape[1]

        # Encoder에서의 과정과 동일하다. target 문장도 임베딩을 진행한다.
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)

        # pos: [batch_size, trg_len]

        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))

        # trg: [batch_size, trg_len, hidden_dim]

        for layer in self.layers:
            # 소스 마스크와 타겟 마스크 모두 사용
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)

        # trg: [batch_size, trg_len, hidden_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

        output = self.fc_out(trg)

        # output: [batch_size, trg_len, output_dim]

        return output, attention

#### **트랜스포머(Transformer) 아키텍처**

* 최종적인 전체 트랜스포머(Transformer) 모델을 정의합니다.
* 입력이 들어왔을 때 앞서 정의한 인코더와 디코더를 거쳐 출력 문장을 생성합니다.

In [ ]:
class Transformer(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, trg_pad_idx, device='cpu'):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    # 소스 문장의 <pad> 토큰에 대하여 마스크(mask) 값을 0으로 설정
    def make_src_mask(self, src):
        # print("make_src_mask() - src.shape :",src.shape)
        # src: [batch_size, src_len]

        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        # print("make_src_mask() - src_mask :",src_mask)
        # src_mask: [batch_size, 1, 1, src_len]
        '''
        토큰의 index가 1인 위치의 값은 0으로, <pad> 토큰이 아닌 의미 있는 토큰의 경우 값이 1로 바뀐다.
        그리고 shape을 [batch_size, 1, 1, src_len] 이렇게 바꾼다.
        '''

        return src_mask

    # 타겟 문장에서 각 단어는 다음 단어가 무엇인지 알 수 없도록(이전 단어만 보도록) 만들기 위해 마스크를 사용
    def make_trg_mask(self, trg):
        # print("make_trg_mask() - trg.shape :",trg.shape)
        # trg: [batch_size, trg_len]

        """ (마스크 예시1)
        1 0 0 0 0
        1 1 0 0 0
        1 1 1 0 0
        1 1 1 0 0
        1 1 1 0 0
        """
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        # trg_pad_mask: [batch_size, 1, 1, trg_len]
        # src_mask처럼 pad 토큰에 해당하는 부분은 의미없는 토큰이기 때문에
        # decoder에서 생성할 필요가 없는 부분이라고 인식시키기 위함.

        trg_len = trg.shape[1]

        """ (마스크 예시2)
        1 0 0 0 0
        1 1 0 0 0
        1 1 1 0 0
        1 1 1 1 0
        1 1 1 1 1
        """
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()
        # trg_sub_mask: [trg_len, trg_len]
        # torch.tril() : 주어진 행렬의 '하삼각 행렬'을 반환한다. +) 하삼각 행렬 = 주대각 성분 위쪽의 모든 성분이 0인 정방행렬
        # bool() : 1,0을 각각 True, False값으로 바꾼다.

        trg_mask = trg_pad_mask & trg_sub_mask
        # torch에서 자동으로 trg_sub_mask를 broadcasting하여 차원을 맞춘다.
        # broadcast : trg_sub_mask의 차원이 (1,1,trg_len,trg_len)으로 바뀐다.
        # &(AND) 연산 : 마지막 차원 trg_len에 대하여 &연산이 이루어진다.
        '''
        마스크 예시1과 마스크 예시2를 AND 연산 한 것이라 생각하면 된다.
        1 0 0 0 0
        1 1 0 0 0
        1 1 1 0 0
        1 1 1 0 0
        1 1 1 0 0
        '''

        # trg_mask: [batch_size, 1, trg_len, trg_len]

        return trg_mask

    def forward(self, src, trg):

        # src: [batch_size, src_len]
        # trg: [batch_size, trg_len]

        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)

        # src_mask: [batch_size, 1, 1, src_len]
        # trg_mask: [batch_size, 1, trg_len, trg_len]

        enc_src = self.encoder(src, src_mask)

        # enc_src: [batch_size, src_len, hidden_dim]

        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)

        # output: [batch_size, trg_len, output_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

        return output, attention

#### **학습(Training)**

* 하이퍼 파라미터 설정 및 모델 초기화

In [ ]:
len(SRC.vocab), len(TRG.vocab)

In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
HIDDEN_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

In [ ]:
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

print(SRC_PAD_IDX)
print(TRG_PAD_IDX)

In [ ]:
# 인코더(encoder)와 디코더(decoder) 객체 선언
enc = Encoder(INPUT_DIM, HIDDEN_DIM, ENC_LAYERS, ENC_HEADS, ENC_PF_DIM, ENC_DROPOUT,device, max_length = max_length+50) # max_length를 넉넉히 설정해야 positional_embedding 시 index 오류가 안 남.
dec = Decoder(OUTPUT_DIM, HIDDEN_DIM, DEC_LAYERS, DEC_HEADS, DEC_PF_DIM, DEC_DROPOUT, device, max_length = max_length+50)

# Transformer 객체 선언
model = Transformer(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

* **모델 가중치 파라미터 초기화**

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

In [ ]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(initialize_weights)

* 학습 및 평가 함수 정의
    * 기본적인 Seq2Seq 모델과 거의 유사하게 작성할 수 있습니다.

In [ ]:
import torch.optim as optim

# Adam optimizer로 학습 최적화
LEARNING_RATE = 0.0005
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# 뒷 부분의 패딩(padding)에 대해서는 값 무시
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
# 모델 학습(train) 함수
def train(model, iterator, optimizer, criterion, clip):
    model.train() # 학습 모드
    epoch_loss = 0

    # 전체 학습 데이터를 확인하며
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()

        # 출력 단어의 마지막 인덱스()는 제외 >> <eos> 토큰을 제외하고 훈련시킴.
        # 📢학습 시 <eos> 토큰을 제외하는 이유 : decoder에서 다음 단어를 예측하는 방식으로 학습하는데 <eos>은 예측할 대상이 아니기 때문이다.
        # 입력을 할 때는 <sos>토큰부터 시작하도록 처리
        # 📢학습 시 <sos> 토큰은 포함해야 하는 이유 : decoder가 첫 단어를 명확히 예측하기 위해서는 <sos>와 encoder의 key,value가 만나서 attention을 수행해야 한다.
        output, _ = model(src, trg[:,:-1])

        # output: [배치 크기, trg_len - 1, output_dim]
        # trg: [배치 크기, trg_len]

        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)
        # 출력 단어의 인덱스 0()은 제외
        trg = trg[:,1:].contiguous().view(-1)

        # output: [배치 크기 * trg_len - 1, output_dim]
        # trg: [배치 크기 * trg len - 1]

        # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
        loss = criterion(output, trg)
        loss.backward() # 기울기(gradient) 계산

        # 기울기(gradient) clipping 진행
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        # 📢gradient clipping : 기울기 폭주(gradient exploding) 문제를 해결하기 위해 사용되는 기법
        # > clip_grad_norm_()함수가 모델 가중치들의 기울기를 모두 하나의 텐서로 묶어 L2-norm을 계산
        # > L2-norm이 설정한 clip 임계값보다 크다면, 기울기의 방향은 유지한 채로 gradient값만 clip값으로 scaling하여 조절한다.
        # > clipping된 기울기로 optimizer가 업데이트를 한다.

        # 파라미터 업데이트
        optimizer.step()

        # 전체 손실 값 계산
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [ ]:
# 모델 평가(evaluate) 함수
def evaluate(model, iterator, criterion):
    model.eval() # 평가 모드
    epoch_loss = 0

    with torch.no_grad():
        # 전체 평가 데이터를 확인하며
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg

            # 출력 단어의 마지막 인덱스(<eos>)는 제외
            # 입력을 할 때는 <sos>부터 시작하도록 처리
            output, _ = model(src, trg[:,:-1])

            # output: [배치 크기, trg_len - 1, output_dim]
            # trg: [배치 크기, trg_len]

            output_dim = output.shape[-1]

            output = output.contiguous().view(-1, output_dim)
            # 출력 단어의 인덱스 0(<sos>)은 제외
            trg = trg[:,1:].contiguous().view(-1)

            # output: [배치 크기 * trg_len - 1, output_dim]
            # trg: [배치 크기 * trg len - 1]

            # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
            loss = criterion(output, trg)

            # 전체 손실 값 계산
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

* 학습(training) 및 검증(validation) 진행
    * **학습 횟수(epoch)**: 100

In [ ]:
import math
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
import time
import math
import random

# 환경 변수 설정
# os.environ['TORCH_USE_CUDA_DSA'] = '1'
# CUDA_LAUNCH_BLOCKING=1

N_EPOCHS = 100
CLIP = 1
best_valid_loss = float('inf')
patience = 0
past_epoch = 0

for epoch in range(N_EPOCHS):
    start_time = time.time() # 시작 시간 기록

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time() # 종료 시간 기록
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        if past_epoch == epoch - 1 :
            patience += 1
        else:
            patience = 0
        if patience == 8:
            torch.save(model.state_dict(), 'transformer_korean_to_english.pt')
            print("Early stopped after 20 epochs. Epoch :",epoch)
            break
    else:
        patience = 0
    past_epoch = epoch

    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')
    print(f'\tValidation Loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):.3f}')

In [ ]:
# # 학습된 모델 저장
# from google.colab import files

# files.download('transformer_german_to_english.pt')

#### **모델 최종 테스트(testing) 결과 확인**

In [ ]:
# !wget https://postechackr-my.sharepoint.com/:u:/g/personal/dongbinna_postech_ac_kr/EbWFiKBmscFBrbzCQxRyqwsBwcXgdKdimkdsBl2dE9VYaQ?download=1 -O transformer_german_to_english.pt

In [ ]:
# model.load_state_dict(torch.load('transformer_german_to_english.pt'))

# test_loss = evaluate(model, test_iterator, criterion)

# print(f'Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):.3f}')

#### **나만의 데이터로 모델 사용해보기**

* 테스트 데이터셋을 이용해 모델 테스트 진행

In [ ]:
# 번역(translation) 함수
def translate_sentence(sentence, src_field, trg_field, model, device, max_len=50, logging=True):
    model.eval() # 평가 모드

    if isinstance(sentence, str):
        nlp = spacy.load('de')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    # 처음에 <sos> 토큰, 마지막에 <eos> 토큰 붙이기
    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
    if logging:
        print(f"전체 소스 토큰: {tokens}")

    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    if logging:
        print(f"소스 문장 인덱스: {src_indexes}")

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)

    # 소스 문장에 따른 마스크 생성
    src_mask = model.make_src_mask(src_tensor)

    # 인코더(endocer)에 소스 문장을 넣어 출력 값 구하기
    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)

    # 처음에는 <sos> 토큰 하나만 가지고 있도록 하기
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):
        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        # 출력 문장에 따른 마스크 생성
        trg_mask = model.make_trg_mask(trg_tensor)

        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)

        # 출력 문장에서 가장 마지막 단어만 사용
        pred_token = output.argmax(2)[:,-1].item()
        trg_indexes.append(pred_token) # 출력 문장에 더하기

        # <eos>를 만나는 순간 끝
        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break

    # 각 출력 단어 인덱스를 실제 단어로 변환
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]

    # 첫 번째 <sos>는 제외하고 출력 문장 반환
    return trg_tokens[1:], attention

In [ ]:
example_idx = 10

src = vars(test_dataset.examples[example_idx])['src']
trg = vars(test_dataset.examples[example_idx])['trg']

print(f'소스 문장: {src}')
print(f'타겟 문장: {trg}')

translation, attention = translate_sentence(src, SRC, TRG, model, device, logging=True)

print("모델 출력 결과:", " ".join(translation))

* 어텐션 맵(Attention Map) 시각화

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib
matplotlib.rcParams['font.family'] = 'Malgun Gothic' # 한글 패치
# ignore warning
import warnings
warnings.filterwarnings("ignore")

def display_attention(sentence, translation, attention, n_heads=8, n_rows=4, n_cols=2):

    assert n_rows * n_cols == n_heads

    # 출력할 그림 크기 조절
    fig = plt.figure(figsize=(15, 25))

    for i in range(n_heads):
        ax = fig.add_subplot(n_rows, n_cols, i + 1)

        # 어텐션(Attention) 스코어 확률 값을 이용해 그리기
        _attention = attention.squeeze(0)[i].cpu().detach().numpy()

        cax = ax.matshow(_attention, cmap='bone')

        ax.tick_params(labelsize=12)
        ax.set_xticklabels([''] + ['<sos>'] + [t.lower() for t in sentence] + ['<eos>'], rotation=45)
        ax.set_yticklabels([''] + translation)

        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

        # color bar 추가
        fig.colorbar(cax, ax=ax)

    plt.show()
    plt.close()

In [ ]:
example_idx = 10

src = vars(test_dataset.examples[example_idx])['src']
trg = vars(test_dataset.examples[example_idx])['trg']

print(f'소스 문장: {src}')
print(f'타겟 문장: {trg}')

translation, attention = translate_sentence(src, SRC, TRG, model, device, logging=True)

print("모델 출력 결과:", " ".join(translation))

In [ ]:
display_attention(src, translation, attention)

#### <b>BLEU Score 계산</b>

* 학습된 트랜스포머(Transformer) 모델의 BLEU 스코어 계산

In [ ]:
from torchtext.data.metrics import bleu_score

def show_bleu(data, src_field, trg_field, model, device, max_len=50):
    trgs = []
    pred_trgs = []
    index = 0

    for datum in data:
        src = vars(datum)['src']
        trg = vars(datum)['trg']

        pred_trg, _ = translate_sentence(src, src_field, trg_field, model, device, max_len, logging=False)

        # 마지막 <eos> 토큰 제거
        pred_trg = pred_trg[:-1]

        pred_trgs.append(pred_trg)
        trgs.append([trg])

        index += 1
        if (index + 1) % 100 == 0:
            print(f"[{index + 1}/{len(data)}]")
            print(f"예측: {pred_trg}")
            print(f"정답: {trg}")

    bleu = bleu_score(pred_trgs, trgs, max_n=4, weights=[0.25, 0.25, 0.25, 0.25])
    print(f'Total BLEU Score = {bleu*100:.2f}')

    individual_bleu1_score = bleu_score(pred_trgs, trgs, max_n=4, weights=[1, 0, 0, 0])
    individual_bleu2_score = bleu_score(pred_trgs, trgs, max_n=4, weights=[0, 1, 0, 0])
    individual_bleu3_score = bleu_score(pred_trgs, trgs, max_n=4, weights=[0, 0, 1, 0])
    individual_bleu4_score = bleu_score(pred_trgs, trgs, max_n=4, weights=[0, 0, 0, 1])

    print(f'Individual BLEU1 score = {individual_bleu1_score*100:.2f}')
    print(f'Individual BLEU2 score = {individual_bleu2_score*100:.2f}')
    print(f'Individual BLEU3 score = {individual_bleu3_score*100:.2f}')
    print(f'Individual BLEU4 score = {individual_bleu4_score*100:.2f}')

    cumulative_bleu1_score = bleu_score(pred_trgs, trgs, max_n=4, weights=[1, 0, 0, 0])
    cumulative_bleu2_score = bleu_score(pred_trgs, trgs, max_n=4, weights=[1/2, 1/2, 0, 0])
    cumulative_bleu3_score = bleu_score(pred_trgs, trgs, max_n=4, weights=[1/3, 1/3, 1/3, 0])
    cumulative_bleu4_score = bleu_score(pred_trgs, trgs, max_n=4, weights=[1/4, 1/4, 1/4, 1/4])

    print(f'Cumulative BLEU1 score = {cumulative_bleu1_score*100:.2f}')
    print(f'Cumulative BLEU2 score = {cumulative_bleu2_score*100:.2f}')
    print(f'Cumulative BLEU3 score = {cumulative_bleu3_score*100:.2f}')
    print(f'Cumulative BLEU4 score = {cumulative_bleu4_score*100:.2f}')

In [ ]:
show_bleu(test_dataset, SRC, TRG, model, device)